<a name='0'></a>
### Import Functions and Data

In [161]:
# run this cell to import nltk
import nltk
from os import getcwd
import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [162]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [163]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

### Prepare the Data


In [164]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [165]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [166]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [167]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [168]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11436


In [169]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


### Training the Model

In [170]:
def sigmoid(z): 
    h = 1/(1+(np.e)**-z)
    
    return h

In [171]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


In [172]:
# Test your function
w1_unittest.test_sigmoid(sigmoid)

 All tests passed


In [173]:
# verify that when the model predicts close to 1, but the actual label is 0, the loss is a large positive value
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

In [174]:
# verify that when the model predicts close to 0 but the actual label is 1, the loss is a large positive value
-1 * np.log(0.0001) # loss is about 9.2

9.210340371976182

In [175]:
def gradientDescent(x, y, theta, alpha, num_iters):

    m = len(y)
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = x@theta
        
        # get the sigmoid of z
        h = 1/(1+(np.e)**-z)
        
        # calculate the cost function
        J = -(1 / m) * np.sum(y.T @ np.log(h) + (1 - y).T @ np.log(1 - h))

        # update the weights theta
        theta = theta - ((alpha/m) *(x.T)@(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [176]:

np.random.seed(1)
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
tmp_Y = ((np.random.rand(10, 1) > 0.35)).astype(float)

tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


In [177]:
# Test your function
w1_unittest.test_gradientDescent(gradientDescent)

 All tests passed


In [178]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements for [bias, positive, negative] counts
    x = np.zeros(3) 
    
    # bias term is set to 1
    x[0] = 1 
        
    # loop through each word in the list of words
    
   
    for word in word_l:
            pair_pos = (word, 1)
            if pair_pos in freqs:
                x[1]+=freqs[pair_pos]
                
            pair_neg = (word, 0)
            if pair_neg in freqs:
                x[2]+=freqs[pair_neg]
        
    ### END CODE HERE ###
    if x.ndim == 1 and x.shape[0] == 3:
         x = x[None, :]  # adding batch dimension for further processing
            
    assert(x.shape == (1, 3))
    return x

In [179]:
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.000e+00 3.133e+03 6.100e+01]]


In [180]:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [181]:
# Test your function
w1_unittest.test_extract_features(extract_features, freqs)

 All tests passed


In [182]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

KeyboardInterrupt: 

In [ ]:
def predict_tweet(tweet, freqs, theta):
        
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs, process_tweet=process_tweet)
    
    # make the prediction using x and theta
    y_pred = sigmoid(x @ theta)
        
    return y_pred

In [ ]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))    
    

In [ ]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

In [ ]:
# Test your function
w1_unittest.test_predict_tweet(predict_tweet, freqs, theta)

### Testing the Model

In [ ]:
import numpy as np

def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    
    # Initialize count variables
    correct_count = 0
    total_count = len(test_x)
    
    # loop through each tweet in the test set
    for tweet, true_label in zip(test_x, test_y):
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        # Convert y_pred to 1.0 if it's greater than 0.5, otherwise 0.0
        predicted_label = 1.0 if y_pred > 0.5 else 0.0
        
        # Compare predicted label to true label and increment correct_count if correct
        if predicted_label == true_label:
            correct_count += 1
    
    # Calculate accuracy as a numpy float64
    accuracy = np.float64(correct_count / total_count)  # Percentage of correct predictions

    return accuracy


In [ ]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

In [ ]:
# Test your function
w1_unittest.unittest_test_logistic_regression(test_logistic_regression, freqs, theta)

In [ ]:
# Some error analysis done for you
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

### Predict your Own Tweet

In [183]:
# Feel free to change the tweet below
my_tweet = 'good boy'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['good', 'boy']
[[0.51446293]]
Positive sentiment
